In [6]:
# import pandas as pd
import tkinter as tk
from tkinter import ttk
# from PIL import Image, ImageOps
# from pillow_heif import register_heif_opener
from docx import Document
from docx.shared import Inches
from docx.shared import Pt
from docx.oxml import OxmlElement
from docx.oxml.ns import qn
from lxml import etree
from docx.enum.table import WD_TABLE_ALIGNMENT
import traceback
from docx.enum.text import WD_ALIGN_PARAGRAPH
import pandas as pd
from docx.enum.text import WD_PARAGRAPH_ALIGNMENT



def format_client(input_text):
    # Split the text at every occurrence of "and" (case-insensitive)
    parts = input_text.split('and')
    
    # Combine parts with "and" followed by a newline after each part
    formatted_text = (' and\n'.join(part.strip() for part in parts)).strip()
    
    return formatted_text

def format_address(address):
    # Split the address into components based on known positions
    parts = address.split()
    
    # Assuming the address format is always correct and has a space between components
    street_address = ' '.join(parts[:-3])  # All but last three elements
    city = parts[-3]
    state = parts[-2]
    zip_code = parts[-1]
    
    # Format the address
    formatted_address = f"{street_address}\n{city} {state} {zip_code}"
    
    return formatted_address

def city_state_zip(address):

    parts = address.split()
    city = parts[-3]
    state = parts[-2]
    zip_code = parts[-1]
    
    # Get city, state, zip
    get_city_state_zip = f"{city} {state} {zip_code}"

    return get_city_state_zip


def extract_street_address(address):
    # Split the address into parts based on spaces
    parts = address.split()
    
    # Ensure there are at least three parts to avoid errors
    if len(parts) < 3:
        raise ValueError("Address must contain at least a street address, city, state, and zip code.")
    
    # Extract the last three parts: city, state, and zip code
    city = parts[-3]
    state = parts[-2]
    zip_code = parts[-1]
    
    # Combine the remaining parts to form the street address
    street_address = ' '.join(parts[:-3])
    
    # Return the street address
    return street_address

def extract_street_name(address):
    # Split the address into parts based on spaces
    parts = address.split()
    
    # Ensure there are at least three parts to avoid errors
    if len(parts) < 3:
        raise ValueError("Address must contain at least a street address, city, state, and zip code.")
    
    # Extract the last three parts: city, state, and zip code
    city = parts[-3]
    state = parts[-2]
    zip_code = parts[-1]
    
    # Combine the remaining parts to form the street address
    street_name = ' '.join(parts[1:-3])
    street_name = street_name.replace(',', '')
    
    # Return the street address
    return street_name

# Define containers for user input
client = input("Enter the client's name: ")
client_mailing_address = input("Enter the client's mailing address (Street Address, County/City, State Zip): ")
property_address = input("Enter the property address  (Street Address, County/City, State Zip): ")
subject_county = input("Enter the county of the subject property: ")
report_date = input("Enter the report date: ")
prg_ref_num = input("Enter the PRG reference number: ")
tax_account = input("Enter the tax account: ")

def lot_size_function():
    print("Please choose a unit of measurement for lot size:")
    print("1. Acres")
    print("2. Square Feet")
    choice = input("Enter the number of your choice (1 or 2): ").strip()
    if choice == "1":
        lot_size = input("Enter the number of acres: ").strip()
        lot_unit = "acres"
    elif choice == "2":
        lot_size = input("Enter the number of lot square feet: ").strip()
        lot_unit = "square feet"
    else:
        print("Invalid choice. Please enter 1 for Acres or 2 for Square Feet.")
    return lot_size, lot_unit  # Return the values

lot_size, lot_unit = lot_size_function()

def parcel_shape_function():
    print("What is the shape of the parcel?")
    print("1. Rectangular")
    print("2. Irregular")
    choice = input("Please enter the number corresponding to your choice: ").strip()
    if choice == "1":
        parcel_shape = "rectangular"
    elif choice == "2":
        parcel_shape = "irregular"
    else:
        print("Invalid choice. Please enter 1 for Rectangular or 2 for Irregular.")
    return parcel_shape

parcel_shape = parcel_shape_function()

def vacant_or_improved():
    print("Is the subject property vacant or improved? ")
    print("1. Vacant")
    print("2. Improved")
    choice = input("Enter the number of your choice: ").strip()
    vac_or_imp = None
    building_sf = None
    year_built = None
    stories = None
    bedrooms = None
    full_bath = None
    half_bath = None
    property_type = None
    if choice == "1":
        vac_or_imp = "vacant"
        print(f"The property is {vac_or_imp}.")
    elif choice == "2":
        vac_or_imp = "improved"
        building_sf = input("Please enter the building square feet: ").strip()
        year_built = input("Enter the year built: ").strip()
        print("Please choose a building type:")
        print("1. Residence")
        print("2. Flex")
        print("3. Mixed-Use")
        building_type_choice = input("Enter the number corresponding to your choice: ").strip()
        if building_type_choice == "1":
            property_type = "residence"
            stories = input("Enter the number of stories: ").strip()
            bedrooms = input("Enter the number of bedrooms: ").strip()
            full_bath = input("Enter the number of full bathrooms: ").strip()
            half_bath = input("Enter the number of half bathrooms: ").strip()
        elif building_type_choice == "2":
            property_type = "flex"
        elif building_type_choice == "3":
            property_type = "mixed-use"
        else:
            print("Invalid choice. Please enter the number corresponding to your choice.")
    return vac_or_imp, building_sf, year_built, stories, bedrooms, full_bath, half_bath, property_type

vac_or_imp, building_sf, year_built, stories, bedrooms, full_bath, half_bath, property_type = vacant_or_improved()
            
liber = input("Enter the deed Liber: ")
folio = input("Enter the deed Folio: ")
map_num = input("Enter the map number: ")
grid = input("Enter the grid number: ")
parcel = input("Enter the parcel number: ")
election_district = input("Enter the election district: ")
census_tract = input("Enter the census tract: ")
road_frontage = input("Enter the total road frontage of the subject property (feet): ")

def choose_value_type():
    global valuation_choice, decedent, date_of_valuation

    # Display the options to the user
    print("Please choose type of valuation:")
    print("1. Current Market Value 'As-Is'")
    print("2. Retrospective Market Value")

    # Loop until a valid choice is made
    while True:
        choice = input("Enter the number of your choice (1 or 2): ").strip()
        if choice == "1":
            # Set the global variables
            valuation_choice = "Current Market Value 'As-Is'"
            decedent = None
            date_of_valuation = input("Enter the date of valuation: ")
            # return valuation_choice
            # return date_of_valuation
            break
        elif choice == "2":
            # Prompt for additional details
            valuation_choice = "Retrospective Market Value"
            decedent = input("Please enter the name of the decedent: ").strip()
            date_of_valuation = input("Please enter the date of death (e.g., YYYY-MM-DD): ").strip()
            # return valuation_choice
            # return decedent
            # return date_of_valuation
            break
        else:
            print("Invalid choice. Please enter 1 for Current Market Value 'As-Is' or 2 for Retrospective Market Value.")

# Call the function to populate the containers
choose_value_type()

def interest_appraised_function():
    print("What is the interest appraised?")
    print("1. Fee Simple")
    print("2. Leased Fee")
    choice = input("Please enter the number corresponding to your choice: ").strip()
    if choice == "1":
        interest_appraised = "Fee Simple"
    elif choice == "2":
        interest_appraised = "Leased Fee"
    else:
        print("Invalid choice. Please enter the number corresponding to your choice.")
    return interest_appraised

interest_appraised = interest_appraised_function()

file_path = "Zoning_Workbook.xlsx"
    
# Read the Excel file into a DataFrame
df = pd.read_excel(file_path)

# Show the values in the first column to the user
first_column_values = df.iloc[:, 0].unique()
print("Please the subject's zoning value from the following list:")
for idx, value in enumerate(first_column_values, start=1):
    print(f"{idx}. {value}")

# Prompt the user to select a value
selection = int(input("\nEnter the number corresponding to your choice: "))
selected_value = first_column_values[selection]

# Create a new DataFrame with the row(s) corresponding to the user's selection
new_df = df[df.iloc[:, 0] == selected_value]\

zone = new_df.iloc[0, 0]
zone_abr = new_df.iloc[0, 1]
ib = new_df.iloc[0, 2]
zone_desc = new_df.iloc[0, 3]

prepared_by = input("Prepared by: ")

# Load the base template
doc = Document('Base Template.docx')

# Set the desired margins (in inches)
left_margin = Inches(1)
right_margin = Inches(1)

# Apply margins to all sections in the document
for section in doc.sections:
    section.left_margin = left_margin
    section.right_margin = right_margin

# Add space before title paragraph
doc.add_paragraph()  # Adds a blank paragraph for spacing

##### Title Page

# Add a title paragraph with centered text
title_paragraph = doc.add_paragraph()
title_run = title_paragraph.add_run("APPRAISAL REPORT")

# Set font properties
title_run.bold = True
title_run.font.size = Pt(36)
title_run.font.name = 'Times New Roman'

# Set paragraph alignment to center
title_paragraph.alignment = 1  # Center alignment

doc.add_paragraph()  # Add a blank paragraph for spacing before the image

# Define the left column values and corresponding user input variables
left_column_values = ["CLIENT", "PROPERTY ADDRESS", "REPORT DATE", "PRG REFERENCE NUMBER"]
right_column_values = [format_client(client), format_address(property_address), report_date, prg_ref_num]

# Function to add a table with one row and two columns
def add_table(doc, left_text, right_text):
    table = doc.add_table(rows=1, cols=2)
    table.autofit = True
    table.style = 'Table Grid'
    
    # Access the first row
    row = table.rows[0]

    # Set column widths
    for cell in row.cells:
        cell.width = Inches(2.5)  # Adjust the width as needed

    # Center alignment of each table
    table.alignment = WD_TABLE_ALIGNMENT.CENTER
    
    # Set the text for the left column
    cell_left = row.cells[0]
    cell_left.text = left_text
    cell_left.paragraphs[0].runs[0].font.size = Pt(12)
    cell_left.paragraphs[0].runs[0].font.name = 'Times New Roman'
    cell_left.paragraphs[0].runs[0].font.bold = True
    cell_left.paragraphs[0].runs[0].text = cell_left.paragraphs[0].runs[0].text.upper()

    # Set the text for the right column
    cell_right = row.cells[1]
    cell_right.text = right_text
    cell_right.paragraphs[0].runs[0].font.size = Pt(12)
    cell_right.paragraphs[0].runs[0].font.name = 'Times New Roman'
    cell_right.paragraphs[0].alignment = WD_ALIGN_PARAGRAPH.RIGHT
    
    # Remove borders from the table
    tbl = table._tbl
    tblPr = tbl.tblPr
    tblBorders = OxmlElement('w:tblBorders')
    for border in ['top', 'left', 'bottom', 'right', 'insideH', 'insideV']:
        border_elem = OxmlElement(f'w:{border}')
        border_elem.set(qn('w:val'), 'nil')
        tblBorders.append(border_elem)
    tblPr.append(tblBorders)
    
    return table

# Populate tables and add them to the document
for i in range(len(left_column_values)):
    add_table(doc, left_column_values[i], right_column_values[i])
    
    # Add a blank paragraph for spacing between tables
    doc.add_paragraph()
    
# table.alignment = WD_TABLE_ALIGNMENT.CENTER

# Add the image after the table
doc.add_paragraph()  # Add a blank paragraph for spacing before the image

# Add the image after the table
doc.add_paragraph()  # Add a blank paragraph for spacing before the image

# # Register HEIC opener with Pillow
# register_heif_opener()

# # Function to convert HEIC to JPG
# def convert_heic_to_jpg(heic_file_path, jpg_file_path):
#     # Open HEIC file using Pillow
#     image = Image.open(heic_file_path)
    
#     # Save as JPG
#     image.save(jpg_file_path, format='JPEG')

# # File paths
# heic_file_path = 'IMG_1303.HEIC'
# jpg_file_path = 'IMG_1303.jpg'

# # Convert HEIC to JPG
# convert_heic_to_jpg(heic_file_path, jpg_file_path)

# # Create a paragraph for centering the image
# image_paragraph = doc.add_paragraph()
# image_paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER  # Center alignment

# # Add the image to the document
# # Resize the image (example size: 5 inches wide by 4 inches high)
# image_run = image_paragraph.add_run().add_picture(jpg_file_path, width=Inches(5), height=Inches(4))

# # Add the image after the table
# doc.add_paragraph()  # Add a blank paragraph for spacing before the image

# Create a new paragraph for the address
address_paragraph = doc.add_paragraph()
address_paragraph.alignment = 1  # Center alignment

# Add address text to the paragraph
address_run = address_paragraph.add_run(property_address)

# Set font properties for the address
address_run.bold = True
address_run.italic = True
address_run.font.size = Pt(12)
address_run.font.name = 'Times New Roman'

# Add the image after the table
doc.add_paragraph()  # Add a blank paragraph for spacing before the image
doc.add_paragraph()

# Add the PRG Logo image to the same paragraph
logo_paragraph = doc.add_paragraph()
logo_paragraph.alignment = 1  # Center alignment

# Add the PRG Logo image to the paragraph
logo_paragraph.add_run().add_picture('PRG Logo.jpg', width=Inches(2.0))  # Adjust width as needed

# Add a page break
doc.add_page_break()



#### Page 2



doc.add_paragraph()  # Adds a blank paragraph for spacing
doc.add_paragraph()  # Adds a blank paragraph for spacing
doc.add_paragraph()  # Adds a blank paragraph for spacing
doc.add_paragraph()  # Adds a blank paragraph for spacing

# Add a title paragraph with centered text
title_paragraph = doc.add_paragraph()
title_run = title_paragraph.add_run("APPRAISAL REPORT")
title_run.bold = True
title_run.font.size = Pt(14)
title_run.font.name = 'Times New Roman'
title_paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER

doc.add_paragraph()  # Adds a blank paragraph for spacing
doc.add_paragraph()  # Adds a blank paragraph for spacing

# Add a title paragraph with centered text
title_paragraph = doc.add_paragraph()
title_run = title_paragraph.add_run("ADDRESS")
title_run.bold = True
title_run.font.size = Pt(12)
title_run.font.name = 'Times New Roman'
title_paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER

doc.add_paragraph()  # Adds a blank paragraph for spacing

# Creates "Address" section
paragraph = doc.add_paragraph(format_address(property_address))
paragraph.alignment = 1

doc.add_paragraph()  # Adds a blank paragraph for spacing
doc.add_paragraph()  # Adds a blank paragraph for spacing

# Add a title paragraph with centered text
legal_desc_paragraph = doc.add_paragraph()
legal_desc_run = legal_desc_paragraph.add_run("LEGAL DESCRIPTION")
legal_desc_run.bold = True
legal_desc_run.font.size = Pt(12)
legal_desc_run.font.name = 'Times New Roman'
legal_desc_paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER

doc.add_paragraph()  # Adds a blank paragraph for spacing

# Creates legal description
par1 = doc.add_paragraph(f"Tax Account: {tax_account}")
par1.alignment = 1
par2 = doc.add_paragraph(f"{lot_size} {lot_unit} of land")
par2.alignment = 1
par3 = doc.add_paragraph(f"Liber {liber}/Folio {folio}")
par3.alignment = 1
par4 = doc.add_paragraph(f"Map {map_num}/Grid {grid}/Parcel {parcel}")
par4.alignment = 1
par5 = doc.add_paragraph(f"{election_district} Election District")
par5.alignment = 1
par6 = doc.add_paragraph(f"Census Tract: {census_tract}")
par6.alignment = 1
par7 = doc.add_paragraph(f"{city_state_zip(property_address)}")
par7.alignment = 1

doc.add_paragraph()  # Adds a blank paragraph for spacing
doc.add_paragraph()  # Adds a blank paragraph for spacing

# Add a title paragraph with centered text
as_of_paragraph = doc.add_paragraph()
as_of_run = as_of_paragraph.add_run("AS OF")
as_of_run.bold = True
as_of_run.font.size = Pt(12)
as_of_run.font.name = 'Times New Roman'
as_of_paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER

doc.add_paragraph()  # Adds a blank paragraph for spacing

# Creates the "As Of" section
par8 = doc.add_paragraph(f"{date_of_valuation}")
par8.alignment = 1
par9 = doc.add_paragraph(f"{valuation_choice}")
par9.alignment = 1

doc.add_paragraph()  # Adds a blank paragraph for spacing
doc.add_paragraph()  # Adds a blank paragraph for spacing

# Add a title paragraph with centered text
as_of_paragraph = doc.add_paragraph()
as_of_run = as_of_paragraph.add_run("FOR")
as_of_run.bold = True
as_of_run.font.size = Pt(12)
as_of_run.font.name = 'Times New Roman'
as_of_paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER

doc.add_paragraph()  # Adds a blank paragraph for spacing

# Creates "For" section
par10 = doc.add_paragraph(f"{client}")
par10.alignment = 1
par11 = doc.add_paragraph(f"{format_address(client_mailing_address)}")
par11.alignment = 1

doc.add_paragraph()  # Adds a blank paragraph for spacing
doc.add_paragraph()  # Adds a blank paragraph for spacing

# Add a title paragraph with centered text
prep_paragraph = doc.add_paragraph()
prep_run = prep_paragraph.add_run("PREPARED BY")
prep_run.bold = True
prep_run.font.size = Pt(12)
prep_run.font.name = 'Times New Roman'
prep_paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER

doc.add_paragraph()  # Adds a blank paragraph for spacing

# Creates the "Prepared By" section
par12 = doc.add_paragraph(f"{prepared_by}")
par12.alignment = 1

doc.add_paragraph()  # Adds a blank paragraph for spacing
doc.add_paragraph()  # Adds a blank paragraph for spacing

# Add a title paragraph with centered text
prep_paragraph = doc.add_paragraph()
prep_run = prep_paragraph.add_run("Copyright © 2024")
prep_run.bold = True
prep_run.font.size = Pt(10)
prep_run.font.name = 'Calibri'
prep_paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER

# Add a title paragraph with centered text
prep_paragraph = doc.add_paragraph()
prep_run = prep_paragraph.add_run("Pugh Real Estate Group, LLC")
prep_run.bold = True
prep_run.font.size = Pt(12)
prep_run.font.name = 'Calibri'
prep_paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER

# Add a title paragraph with centered text
prep_paragraph = doc.add_paragraph()
prep_run = prep_paragraph.add_run("Real Estate Appraisers")
prep_run.bold = True
prep_run.font.size = Pt(10)
prep_run.font.name = 'Calibri'
prep_paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER

# Add a title paragraph with centered text
prep_paragraph = doc.add_paragraph()
prep_run = prep_paragraph.add_run("P.O. Box 378")
prep_run.bold = True
prep_run.font.size = Pt(10)
prep_run.font.name = 'Calibri'
prep_paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER

# Add a title paragraph with centered text
prep_paragraph = doc.add_paragraph()
prep_run = prep_paragraph.add_run("Frederick, Maryland 21705")
prep_run.bold = True
prep_run.font.size = Pt(10)
prep_run.font.name = 'Calibri'
prep_paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER

# Add a page break
doc.add_page_break()



#### Page 3



# Add a title paragraph with centered text
prep_paragraph = doc.add_paragraph()

# "PUGH"
prep_run_p = prep_paragraph.add_run("P")
prep_run_p.font.size = Pt(18)
prep_run_p.font.name = 'Copperplate Gothic Bold'

prep_run_ugh = prep_paragraph.add_run("UGH")
prep_run_ugh.font.size = Pt(16)
prep_run_ugh.font.name = 'Copperplate Gothic Bold'

# "REAL"
prep_run_r = prep_paragraph.add_run(" R")
prep_run_r.font.size = Pt(18)
prep_run_r.font.name = 'Copperplate Gothic Bold'

prep_run_eal = prep_paragraph.add_run("EAL")
prep_run_eal.font.size = Pt(16)
prep_run_eal.font.name = 'Copperplate Gothic Bold'

# "ESTATE"
prep_run_r = prep_paragraph.add_run(" E")
prep_run_r.font.size = Pt(18)
prep_run_r.font.name = 'Copperplate Gothic Bold'

prep_run_estate = prep_paragraph.add_run("STATE")
prep_run_estate.font.size = Pt(16)
prep_run_estate.font.name = 'Copperplate Gothic Bold'

# "GROUP"
prep_run_g = prep_paragraph.add_run(" G")
prep_run_g.font.size = Pt(18)
prep_run_g.font.name = 'Copperplate Gothic Bold'

prep_run_roup = prep_paragraph.add_run("ROUP,")
prep_run_roup.font.size = Pt(16)
prep_run_roup.font.name = 'Copperplate Gothic Bold'

# LLC
# "GROUP"
prep_run_g = prep_paragraph.add_run(" LLC")
prep_run_g.font.size = Pt(18)
prep_run_g.font.name = 'Copperplate Gothic Bold'

# Align the paragraph to center
prep_paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER

# Add a title paragraph with centered text
prep_paragraph = doc.add_paragraph()
prep_run = prep_paragraph.add_run("REAL ESTATE APPRAISERS")
prep_run.bold = False
prep_run.font.size = Pt(10)
prep_run.font.name = 'Copperplate Gothic Bold'
prep_paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER

# Add a title paragraph with centered text
prep_paragraph = doc.add_paragraph()
prep_run = prep_paragraph.add_run("POST OFFICE BOX 378")
prep_run.bold = False
prep_run.font.size = Pt(6)
prep_run.font.name = 'Copperplate Gothic Bold'
prep_paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER

# Add a title paragraph with centered text
prep_paragraph = doc.add_paragraph()
prep_run = prep_paragraph.add_run("FREDERICK, MARYLAND 21705")
prep_run.bold = False
prep_run.font.size = Pt(6)
prep_run.font.name = 'Copperplate Gothic Bold'
prep_paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER

# Add a title paragraph with centered text
prep_paragraph = doc.add_paragraph()
prep_run = prep_paragraph.add_run("(301) 898-1178 ")
prep_run.bold = False
prep_run.font.size = Pt(8)
prep_run.font.name = 'Copperplate Gothic Bold'
prep_run.font.underline = True 
prep_paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER

doc.add_paragraph()  # Adds a blank paragraph for spacing

# Add a title paragraph with centered text
prep_paragraph = doc.add_paragraph()
prep_run = prep_paragraph.add_run(f"{report_date}")
prep_run.bold = False
prep_run.font.size = Pt(12)
prep_run.font.name = 'Times New Roman'
prep_paragraph.alignment = WD_ALIGN_PARAGRAPH.RIGHT

# Add a title paragraph with centered text
prep_paragraph = doc.add_paragraph()
prep_run = prep_paragraph.add_run(f"{client}")
prep_run.bold = False
prep_run.font.size = Pt(12)
prep_run.font.name = 'Times New Roman'
prep_paragraph.alignment = WD_ALIGN_PARAGRAPH.LEFT

# Add a title paragraph with centered text
prep_paragraph = doc.add_paragraph()
prep_run = prep_paragraph.add_run(f"{format_address(client_mailing_address)}")
prep_run.bold = False
prep_run.font.size = Pt(12)
prep_run.font.name = 'Times New Roman'
prep_paragraph.alignment = WD_ALIGN_PARAGRAPH.LEFT

doc.add_paragraph()  # Adds a blank paragraph for spacing

# Add a title paragraph with centered text
prep_paragraph = doc.add_paragraph()
prep_run = prep_paragraph.add_run(f"Re:     {extract_street_address(property_address)}")
prep_run.bold = False
prep_run.font.size = Pt(12)
prep_run.font.name = 'Times New Roman'
prep_paragraph.alignment = WD_ALIGN_PARAGRAPH.LEFT

# Add a title paragraph with centered text
prep_paragraph = doc.add_paragraph()
prep_run = prep_paragraph.add_run(f"           {city_state_zip(property_address)}")
prep_run.bold = False
prep_run.font.size = Pt(12)
prep_run.font.name = 'Times New Roman'
prep_paragraph.alignment = WD_ALIGN_PARAGRAPH.LEFT

doc.add_paragraph()  # Adds a blank paragraph for spacing

# Add a title paragraph with centered text
prep_paragraph = doc.add_paragraph()
prep_run = prep_paragraph.add_run(f"Dear {client},")
prep_run.bold = False
prep_run.font.size = Pt(12)
prep_run.font.name = 'Times New Roman'
prep_paragraph.alignment = WD_ALIGN_PARAGRAPH.LEFT

doc.add_paragraph()  # Adds a blank paragraph for spacing

def add_paragraph_based_on_choice():
    global valuation_choice, decedent, date_of_death, report_date

    # Add the title paragraph with centered text based on the valuation choice
    if valuation_choice == "Current Market Value 'As-Is'":
        # Add a title paragraph with centered text
        prep_paragraph = doc.add_paragraph()
        prep_run = prep_paragraph.add_run(
            f"""You asked me to provide you with an appraisal value for the property identified above. This appraisal report provides an opinion of value as of {report_date}, which was the date of inspection. The attached report provides essential data and detailed reasoning employed in reaching my opinion of value."""
        )
        prep_run.bold = False
        prep_run.font.size = Pt(12)
        prep_run.font.name = 'Times New Roman'
        prep_paragraph.alignment = WD_ALIGN_PARAGRAPH.LEFT
        prep_paragraph.paragraph_format.space_before = Pt(6)
        prep_paragraph.paragraph_format.space_after = Pt(6)


    elif valuation_choice == "Retrospective Market Value":
        # Add a title paragraph with centered text
        prep_paragraph = doc.add_paragraph()
        prep_run = prep_paragraph.add_run(
            f"""Recently, you asked me to provide you with a retrospective market value for the property identified above. This appraisal report provides an opinion of value as of {date_of_death}, which is the date of death of {decedent}. The attached appraisal report provides essential data and detailed reasoning employed in reaching my opinion of value."""
        )
        prep_run.bold = False
        prep_run.font.size = Pt(12)
        prep_run.font.name = 'Times New Roman'
        prep_paragraph.alignment = WD_ALIGN_PARAGRAPH.LEFT
        prep_paragraph.paragraph_format.space_before = Pt(6)
        prep_paragraph.paragraph_format.space_after = Pt(6)


add_paragraph_based_on_choice()

prep_paragraph = doc.add_paragraph()
prep_run = prep_paragraph.add_run("SUMMARY OF SITE, ZONING, AND IMPROVEMENTS")
prep_run.bold = True
prep_run.font.size = Pt(12)
prep_run.font.name = 'Times New Roman'
prep_paragraph.alignment = WD_ALIGN_PARAGRAPH.LEFT
prep_paragraph.paragraph_format.space_before = Pt(6)
prep_paragraph.paragraph_format.space_after = Pt(6)

prep_paragraph = doc.add_paragraph()
prep_run = prep_paragraph.add_run(f"""The subject property is located on a {parcel_shape}-shaped parcel ({parcel}) that encompasses {lot_size} {lot_unit}, according to the land records of {subject_county} County. This site features approximately {road_frontage} feet of frontage along {extract_street_name(property_address)}.""")
prep_run.bold = False
prep_run.font.size = Pt(12)
prep_run.font.name = 'Times New Roman'
prep_paragraph.alignment = WD_ALIGN_PARAGRAPH.LEFT
prep_paragraph.paragraph_format.space_before = Pt(6)
prep_paragraph.paragraph_format.space_after = Pt(6)


def add_paragraph_based_on_property_type():
    global stories, bedrooms, full_bath, half_bath, property_type, building_sf, year_built

    if property_type == "residence" and half_bath not in ["zero",0,"0","Zero"]:
        prep_paragraph = doc.add_paragraph()
        prep_run = prep_paragraph.add_run(
            f"""This parcel includes a {stories}-story residence building with {bedrooms} bedrooms, {full_bath} full bathrooms and {half_bath} half bathrooms. The building has {building_sf} of gross living area and was constructed in {year_built} (according to the {subject_county} county land records).""" 
        )
        prep_run.bold = False
        prep_run.font.size = Pt(12)
        prep_run.font.name = 'Times New Roman'
        prep_paragraph.alignment = WD_ALIGN_PARAGRAPH.LEFT
        prep_paragraph.paragraph_format.space_before = Pt(6)
        prep_paragraph.paragraph_format.space_after = Pt(6)


    elif valuation_choice == "residence" and half_bath in ["zero",0,"0","Zero"]:
        prep_paragraph = doc.add_paragraph()
        prep_run = prep_paragraph.add_run(
            f"""This parcel includes a {stories}-story residence building with {bedrooms} bedrooms and {full_bath} bathrooms. The building has {building_sf} of gross living area and was constructed in {year_built} (according to the {subject_county} county land records.)"""
        )
        prep_run.bold = False
        prep_run.font.size = Pt(12)
        prep_run.font.name = 'Times New Roman'
        prep_paragraph.alignment = WD_ALIGN_PARAGRAPH.LEFT
        prep_paragraph.paragraph_format.space_before = Pt(6)
        prep_paragraph.paragraph_format.space_after = Pt(6)


    else:
        prep_paragraph = doc.add_paragraph()
        prep_run = prep_paragraph.add_run(
            "FILLER PARAGRAPH"
        )
        prep_run.bold = False
        prep_run.font.size = Pt(12)
        prep_run.font.name = 'Times New Roman'
        prep_paragraph.alignment = WD_ALIGN_PARAGRAPH.LEFT
        prep_paragraph.paragraph_format.space_before = Pt(6)
        prep_paragraph.paragraph_format.space_after = Pt(6)


add_paragraph_based_on_property_type()

prep_paragraph = doc.add_paragraph()
prep_run = prep_paragraph.add_run(f"The subject property is zoned {zone} ({zone_abr}) by the {ib}. {zone_desc}")
prep_run.bold = False
prep_run.font.size = Pt(12)
prep_run.font.name = 'Times New Roman'
prep_paragraph.alignment = WD_ALIGN_PARAGRAPH.LEFT
prep_paragraph.paragraph_format.space_before = Pt(6)
prep_paragraph.paragraph_format.space_after = Pt(6)


# Add a page break
doc.add_page_break()



#### Page 4






# First paragraph
prep_paragraph = doc.add_paragraph()
prep_run = prep_paragraph.add_run("EXTRAORDINARY ASSUMPTIONS")
prep_run.bold = True
prep_run.font.size = Pt(12)
prep_run.font.name = 'Times New Roman'
prep_paragraph.alignment = WD_ALIGN_PARAGRAPH.LEFT
prep_paragraph.paragraph_format.space_after = Pt(6)

# Second paragraph
prep_paragraph = doc.add_paragraph()
prep_run = prep_paragraph.add_run("Uniform Standard of Professional Appraisal Practice (USPAP) defines an Extraordinary Assumption as, “an assumption, directly related to a specific assignment, as of the effective date of the assignment result, which, if found to be false, could alter the appraiser’s opinions or conclusions.” ")
prep_run.bold = False
prep_run.font.size = Pt(12)
prep_run.font.name = 'Times New Roman'
prep_paragraph.alignment = WD_ALIGN_PARAGRAPH.LEFT
prep_paragraph.paragraph_format.space_before = Pt(6)
prep_paragraph.paragraph_format.space_after = Pt(6)

# Third paragraph
prep_paragraph = doc.add_paragraph()
prep_run = prep_paragraph.add_run("No extraordinary assumptions were made for this assignment.")
prep_run.bold = False
prep_run.font.size = Pt(12)
prep_run.font.name = 'Times New Roman'
prep_paragraph.alignment = WD_ALIGN_PARAGRAPH.LEFT
prep_run.italic = True
prep_paragraph.paragraph_format.space_before = Pt(6)
prep_paragraph.paragraph_format.space_after = Pt(6)

# Fourth paragraph
prep_paragraph = doc.add_paragraph()
prep_run = prep_paragraph.add_run("HYPOTHETICAL CONDITIONS")
prep_run.bold = True
prep_run.font.size = Pt(12)
prep_run.font.name = 'Times New Roman'
prep_paragraph.alignment = WD_ALIGN_PARAGRAPH.LEFT
prep_paragraph.paragraph_format.space_before = Pt(6)
prep_paragraph.paragraph_format.space_after = Pt(6)

# Fifth paragraph
prep_paragraph = doc.add_paragraph()
prep_run = prep_paragraph.add_run("USPAP defines a Hypothetical Condition as, “that which is contrary to what is known the appraiser to exist on the effective date of the assignment results but is used for the purpose of analysis.” ")
prep_run.bold = False
prep_run.font.size = Pt(12)
prep_run.font.name = 'Times New Roman'
prep_paragraph.alignment = WD_ALIGN_PARAGRAPH.LEFT
prep_paragraph.paragraph_format.space_before = Pt(6)
prep_paragraph.paragraph_format.space_after = Pt(6)

# Sixth paragraph
prep_paragraph = doc.add_paragraph()
prep_run = prep_paragraph.add_run("No hypothetical conditions were made for this assignment.")
prep_run.bold = False
prep_run.font.size = Pt(12)
prep_run.font.name = 'Times New Roman'
prep_paragraph.alignment = WD_ALIGN_PARAGRAPH.LEFT
prep_run.italic = True
prep_paragraph.paragraph_format.space_before = Pt(6)
prep_paragraph.paragraph_format.space_after = Pt(6)

# Seventh paragraph
prep_paragraph = doc.add_paragraph()
prep_run = prep_paragraph.add_run("After careful consideration of all the factors which create value, it is my opinion that the subject property has a value as follows:")
prep_run.bold = False
prep_run.font.size = Pt(12)
prep_run.font.name = 'Times New Roman'
prep_paragraph.alignment = WD_ALIGN_PARAGRAPH.LEFT
prep_paragraph.paragraph_format.space_before = Pt(6)
prep_paragraph.paragraph_format.space_after = Pt(6)

doc.add_paragraph()  # Adds a blank paragraph for spacing

def set_cell_borders(cell, top_color="000000", bottom_color="000000", left_color="000000", right_color="000000", border_width=4):
    """Set the borders for a cell with individual colors for top, bottom, left, and right borders."""
    tc_pr = cell._element.xpath('.//w:tcPr')
    if not tc_pr:
        tc_pr = OxmlElement('w:tcPr')
        cell._element.insert(0, tc_pr)
    else:
        tc_pr = tc_pr[0]

    borders = OxmlElement('w:tcBorders')
    border_attrs = {
        'w:top': top_color,
        'w:left': left_color,
        'w:bottom': bottom_color,
        'w:right': right_color,
        'w:insideH': 'FFFFFF',  # inside borders color, typically set to white
        'w:insideV': 'FFFFFF'   # inside borders color, typically set to white
    }

    for border, color in border_attrs.items():
        border_elem = OxmlElement(border)
        border_elem.set(qn('w:val'), 'single')
        border_elem.set(qn('w:sz'), str(border_width))  # Border width (1/8 pt)
        border_elem.set(qn('w:space'), '0')
        border_elem.set(qn('w:color'), color)  # Set border color
        borders.append(border_elem)

    tc_pr.append(borders)

def set_cell_fill(cell, color):
    """Set the fill color for a cell."""
    shading = OxmlElement('w:shd')
    shading.set(qn('w:fill'), color)
    cell._element.get_or_add_tcPr().append(shading)

# Add a table with 2 rows and 4 columns
table = doc.add_table(rows=2, cols=4)

# Define the headers for the first row
headers_1 = ["APPRAISAL PREMISE", "INTEREST APPRAISED", "DATE OF VALUE", "VALUE CONCLUSION"]
headers_2 = [f"{valuation_choice}", f"{interest_appraised}", f"{date_of_valuation}", "$"]

# Populate the header row with formatting
for col, header in enumerate(headers_1):
    cell = table.cell(0, col)
    cell.text = header
    set_cell_borders(cell, top_color="FFFFFF", bottom_color="000000", left_color="FFFFFF", right_color="FFFFFF", border_width=4)
    # Set text to bold
    for paragraph in cell.paragraphs:
        run = paragraph.runs[0]
        run.bold = True
        run.font.name = "Calibri Light"
        run.font.size = Pt(13)
        paragraph.paragraph_format.space_after = Pt(0)  # Set space after to 0
    cell.paragraphs[0].alignment = WD_PARAGRAPH_ALIGNMENT.CENTER  # Center align the header

# Populate the second row with data from headers_2
for col, header in enumerate(headers_2):
    cell = table.cell(1, col)  # Use row 1 (second row)
    cell.text = header
    if col == 0:
        # For the first cell of the second row, set the right border color to "FFFFFF"
        set_cell_borders(cell, top_color="000000", bottom_color="FFFFFF", left_color="FFFFFF", right_color="000000", border_width=4)
    else:
        set_cell_borders(cell, top_color="000000", bottom_color="FFFFFF", left_color="FFFFFF", right_color="BFBFBF", border_width=4)
        set_cell_fill(cell, color="BFBFBF")
    
    # Set text formatting
    for paragraph in cell.paragraphs:
        if col == 0:
            run = paragraph.runs[0]
            run.font.name = "Times New Roman"
            run.font.size = Pt(12)
            run.font.italic = True
            paragraph.paragraph_format.space_after = Pt(0)  # Set space after to 0
        else:
            run = paragraph.runs[0]
            run.font.name = "Times New Roman"
            run.font.size = Pt(12)
            run.font.italic = False
            paragraph.paragraph_format.space_after = Pt(0)  # Set space after to 0
    cell.paragraphs[0].alignment = WD_PARAGRAPH_ALIGNMENT.CENTER  # Center align the content
    

# Seventh paragraph
prep_paragraph = doc.add_paragraph()
prep_run = prep_paragraph.add_run("If you have any questions concerning this appraisal or the value conclusions developed in this report, please do not hesitate to contact me.")
prep_run.bold = False
prep_run.font.size = Pt(12)
prep_run.font.name = 'Times New Roman'
prep_paragraph.alignment = WD_ALIGN_PARAGRAPH.LEFT
prep_paragraph.paragraph_format.space_before = Pt(6)
prep_paragraph.paragraph_format.space_after = Pt(6)

doc.add_paragraph()  # Adds a blank paragraph for spacing

# Seventh paragraph
prep_paragraph = doc.add_paragraph()
prep_run = prep_paragraph.add_run("Sincerely,")
prep_run.bold = False
prep_run.font.size = Pt(12)
prep_run.font.name = 'Times New Roman'
prep_paragraph.alignment = WD_ALIGN_PARAGRAPH.LEFT
prep_paragraph.paragraph_format.space_before = Pt(6)
prep_paragraph.paragraph_format.space_after = Pt(6)

# Add a page break
doc.add_page_break()




#### Page 5









#### Page 6





def set_paragraph_background_and_border(paragraph, fill_color="A6A6A6", border_color="000000"):
    p = paragraph._element
    pPr = p.find(qn('w:pPr'))
    if pPr is None:
        pPr = OxmlElement('w:pPr')
        p.append(pPr)

    shd = OxmlElement('w:shd')
    shd.set(qn('w:val'), 'clear')
    shd.set(qn('w:color'), 'auto')
    shd.set(qn('w:fill'), fill_color)
    pPr.append(shd)

    for border_type in ['top', 'bottom', 'left', 'right']:
        border = OxmlElement(f'w:{border_type}')
        border.set(qn('w:val'), 'single')
        border.set(qn('w:sz'), '4')
        border.set(qn('w:space'), '0')
        border.set(qn('w:color'), border_color)
        pPr.append(border)

def set_cell_paragraph_formatting(paragraph):
    paragraph_format = paragraph.paragraph_format
    paragraph_format.space_before = Pt(6)

def set_column_width(cell, width):
    cell.width = Inches(width)
    tcPr = cell._element.get_or_add_tcPr()
    tcW = OxmlElement('w:tcW')
    tcW.set(qn('w:w'), str(int(width * 1440)))  # width in twips (1 inch = 1440 twips)
    tcW.set(qn('w:type'), 'dxa')
    tcPr.append(tcW)


paragraph = doc.add_paragraph()
run = paragraph.add_run("SUMMARY OF IMPORTANT FACTS AND CONCLUSIONS")
run.bold = True
run.font.name = 'Times New Roman'
run.font.size = Pt(14)
paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER
set_paragraph_background_and_border(paragraph, fill_color="A6A6A6", border_color="000000")

# Define the table data for the first column
left_column_values1 = [
    "Address", "Date of Value Estimate", "Date of Appraisal Report",
    "Property Rights Appraised", "Land Area", "Improvements", 
    "Zoning", "Intended Use", "Client/Intended User"
]

# Define the table data for the right column
right_column_values = [
    f"{format_address(client_mailing_address)}", 
    f"{date_of_valuation}", 
    f"{report_date}",
    f"{interest_appraised}",
    f"{lot_size}", 
    "", 
    f"The subject property is zoned {zone} ({zone_abr}) by the {ib}.", 
    "", 
    f"{client}"
]

# Add a 9x2 table beneath the previous content
table = doc.add_table(rows=9, cols=2)

# Adjust column widths
left_column_width = 1.5  # Width for the left column in inches
right_column_width = 5  # Width for the right column in inches

# Loop through each row to add the left column and right column text with formatting
for idx in range(len(left_column_values1)):
    # Add value to the first column
    left_cell = table.cell(idx, 0)
    left_paragraph = left_cell.paragraphs[0]
    run = left_paragraph.add_run(left_column_values1[idx])
    run.bold = True
    run.font.name = 'Times New Roman'
    run.font.size = Pt(12)
    set_cell_paragraph_formatting(left_paragraph)
    
    # Set width for the left column
    set_column_width(left_cell, left_column_width)
    
    # Add value to the second column
    right_cell = table.cell(idx, 1)
    right_paragraph = right_cell.paragraphs[0]
    right_run = right_paragraph.add_run(right_column_values[idx])
    right_run.font.name = 'Times New Roman'
    right_run.font.size = Pt(12)
    set_cell_paragraph_formatting(right_paragraph)
    
    # Set width for the second column
    set_column_width(right_cell, right_column_width)
    
paragraph = doc.add_paragraph()
run = paragraph.add_run("HIGHEST AND BEST USE")
run.bold = True
run.font.name = 'Times New Roman'
run.font.size = Pt(14)
paragraph.alignment = WD_ALIGN_PARAGRAPH.LEFT
paragraph.paragraph_format.space_before = Pt(6)
set_paragraph_background_and_border(paragraph, fill_color="A6A6A6", border_color="000000")

left_column_values2 = ["As Vacant", "As Improved"]

table = doc.add_table(rows=2, cols=2)
    
for idx in range(len(left_column_values2)):
    # Add value to the first column
    left_cell = table.cell(idx, 0)
    left_paragraph = left_cell.paragraphs[0]
    run = left_paragraph.add_run(left_column_values2[idx])
    run.bold = True
    run.font.name = 'Times New Roman'
    run.font.size = Pt(12)
    set_cell_paragraph_formatting(left_paragraph)
    
    # Set width for the left column
    set_column_width(left_cell, left_column_width)
    


# # Save the document
doc.save("appraisal_report.docx")

Enter the client's name: lksj
Enter the client's mailing address (Street Address, County/City, State Zip): lksjdg lwkej lsfkgj lwekrj lsfkgj
Enter the property address  (Street Address, County/City, State Zip): ;lsakd wlrktj fslgkj wlekrj elrtjk
Enter the county of the subject property: as;lkgfj
Enter the report date: ldskfjg
Enter the PRG reference number: lkrj
Enter the tax account: lfkdjg
Please choose a unit of measurement for lot size:
1. Acres
2. Square Feet
Enter the number of your choice (1 or 2): 1
Enter the number of acres: 23
What is the shape of the parcel?
1. Rectangular
2. Irregular
Please enter the number corresponding to your choice: 1
Is the subject property vacant or improved? 
1. Vacant
2. Improved
Enter the number of your choice: 1
The property is vacant.
Enter the deed Liber: lksfjg
Enter the deed Folio: elrkt
Enter the map number: lkdfgj
Enter the grid number: skfgj
Enter the parcel number: wlkrtj
Enter the election district: ldkfgj
Enter the census tract: wlekjt
